<a href="https://colab.research.google.com/github/Gaeli75/Deep-Learning-Pratices-1/blob/main/DP_learning_week_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
!pip install lightning-utilities


In [39]:
!pip install torchmetrics --no-deps

In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torchvision
from torchvision.transforms import ToTensor
from torchmetrics import MeanMetric, Accuracy
from torchmetrics import ConfusionMatrix, Accuracy, Precision, Recall, F1Score

In [41]:
device = "cuda" if torch.cuda.is_available()\
     else "mps" if torch.mps.is_available()\
     else "cpu"
print("device", device)

device cuda


In [42]:
#Importing Data for Training Data
train_data = torchvision.datasets.CIFAR10( root = './data', train = True,
    transform = ToTensor(),
    download = True,
)

Files already downloaded and verified


In [43]:
train_data

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [44]:
test_data = torchvision.datasets.CIFAR10( root = './data', train = False,
    transform = ToTensor(),
    download = True,
)

Files already downloaded and verified


In [45]:
test_data

Dataset CIFAR10
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [46]:
train_set, val_set = torch.utils.data.random_split(train_data, [0.8, 0.2])

In [47]:
train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_set, batch_size=32, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True)

In [48]:
model = nn.Sequential(
    nn.LazyConv2d(64, kernel_size=3, stride = 1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size = 2),
    nn.LazyConv2d(128, kernel_size=3, stride =1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Flatten(),
    nn.LazyLinear(512),
    nn.ReLU(), # Changed nn.ReLu() to nn.ReLU()
    nn.LazyLinear(256),
    nn.ReLU(), # Changed nn.ReLu() to nn.ReLU()
    nn.LazyLinear(10),
)

In [49]:
model = model.to(device)

In [50]:
criterian = torch.nn.CrossEntropyLoss()


In [51]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.005)

In [52]:
#Training Model
def train_one_epoch():
    #preparing the data
    losses = MeanMetric().to(device)
    acc  = Accuracy(task = 'multiclass', num_classes = 10).to(device)
    model.train()
    #Loop for the model
    for X, Y in train_dataloader:
        X, Y = X.to(device), Y.to(device)
        optimizer.zero_grad()
        pred = model(X)
        loss = criterian(pred, Y)
        loss.backward()
        optimizer.step()
        preds = pred.argmax(dim = 1)
        losses.update(loss.item(), X.size(0)) # Changed loss.X.size(0) to loss.item(), X.size(0)
        acc.update(preds, Y) # Changed pred to preds, as preds contains the predicted labels

    return losses.compute().item(), acc.compute().item()


In [53]:
#Validation Model
def Validation_one_epoch():
    #preparing the data
    losses = MeanMetric().to(device)
    acc  = Accuracy(task = 'multiclass', num_classes = 10).to(device)
    model.eval()
    #Loop for the model
    for X, Y in val_dataloader:
        X, Y = X.to(device), Y.to(device)
        optimizer.zero_grad()
        pred = model(X)
        loss = criterian(pred, Y)
        loss.backward()
        optimizer.step()
        preds = pred.argmax(dim = 1)
        losses.update(loss.item(), X.size(0)) # Changed loss.X.size(0) to loss.item(), X.size(0)
        acc.update(preds, Y) # Changed pred to preds, as preds contains the predicted labels

    return losses.compute().item(), acc.compute().item()

In [ ]:
epochs = 20
for i in range(0, epochs):
    train_loss, train_acc = train_one_epoch()
    val_loss, val_acc = Validation_one_epoch()
    print("Epoch: ", i, "| Train Loss: ", train_loss, "| Train Accuracy", train_acc, "| Val Loss: ", val_loss, "| Val Accuracy: ", val_acc)

Epoch:  0 | Train Loss:  2.2875993251800537 | Train Accuracy 0.15182499587535858 | Val Loss:  2.2194409370422363 | Val Accuracy:  0.21739999949932098
Epoch:  1 | Train Loss:  2.0442049503326416 | Train Accuracy 0.25687500834465027 | Val Loss:  1.9510042667388916 | Val Accuracy:  0.29589998722076416
Epoch:  2 | Train Loss:  1.867126226425171 | Train Accuracy 0.33434998989105225 | Val Loss:  1.7824804782867432 | Val Accuracy:  0.3617999851703644
Epoch:  3 | Train Loss:  1.6931638717651367 | Train Accuracy 0.39195001125335693 | Val Loss:  1.6200928688049316 | Val Accuracy:  0.41670000553131104
Epoch:  4 | Train Loss:  1.5571194887161255 | Train Accuracy 0.43744999170303345 | Val Loss:  1.5101985931396484 | Val Accuracy:  0.4487999975681305
Epoch:  5 | Train Loss:  1.4658030271530151 | Train Accuracy 0.46882501244544983 | Val Loss:  1.4320054054260254 | Val Accuracy:  0.484499990940094
Epoch:  6 | Train Loss:  1.3927955627441406 | Train Accuracy 0.4984999895095825 | Val Loss:  1.3700691461

In [ ]:
#Storing Evaluation metrics
test_confusion_matrix = ConfusionMatrix(task = 'multiclass', num_classes = 10).to(device) # Moved the confusion matrix to the device
test_accuracy = Accuracy(task = 'multiclass', num_classes = 10).to(device) # Moved the accuracy metric to the device

In [ ]:
model = model.to(device)
model.eval()
with torch.no_grad():
    for X, Y in test_dataloader:
        X, Y = X.to(device), Y.to(device)
        pred = model(X)
        preds = pred.argmax(dim = 1)
        test_confusion_matrix.update(pred, Y)
        test_accuracy.update(pred, Y)
print("Confusion Matrix: \n", test_confusion_matrix.compute())
print("Accuracy: ", test_accuracy.compute().item())